## Importing libraries

In [95]:
import pandas as pd
import numpy as np

import xgboost as xgb

In [46]:
# !pip install xgboost

## Loading data

In [2]:
df = pd.read_csv('historical_.routes.txt', sep=",")

In [3]:
df.head()

,DAY_ID,ROUTE_ID,STORE_NUMBER,DAY_OF_WEEK,HELPER,TOTAL_WEIGHT,CASES,URBANITY,UNLOAD_TIME
0,1,1,176,1,False,76.759312,16,U,46.579371
1,4,1,176,4,False,54.218237,11,U,32.698010
2,5,2,176,5,False,80.911549,17,U,49.060463
3,6,5,176,6,False,65.970720,14,U,35.219056
4,8,4,176,1,False,60.334548,13,U,38.351855


## Pre-modeling Data Analysis

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15321 entries, 0 to 15320
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DAY_ID        15321 non-null  int64  
 1   ROUTE_ID      15321 non-null  int64  
 2   STORE_NUMBER  15321 non-null  int64  
 3   DAY_OF_WEEK   15321 non-null  int64  
 4   HELPER        15321 non-null  bool   
 5   TOTAL_WEIGHT  15321 non-null  float64
 6   CASES         15321 non-null  int64  
 7   URBANITY      15321 non-null  object 
 8   UNLOAD_TIME   15321 non-null  float64
dtypes: bool(1), float64(2), int64(5), object(1)
memory usage: 972.6+ KB


In [5]:
df.describe()

,DAY_ID,ROUTE_ID,STORE_NUMBER,DAY_OF_WEEK,TOTAL_WEIGHT,CASES,UNLOAD_TIME
count,15321.000000,15321.000000,15321.000000,15321.000000,15321.000000,15321.000000,15321.000000
mean,90.229358,4.173422,90.117486,2.998695,55.872456,11.668821,29.163737
std,51.672604,2.251402,51.981693,1.951534,12.720365,2.560516,9.065642
min,1.000000,1.000000,1.000000,0.000000,10.000000,2.000000,6.000000
25%,45.000000,2.000000,45.000000,1.000000,47.347980,10.000000,22.653633
50%,90.000000,4.000000,90.000000,3.000000,55.905264,12.000000,29.066965
75%,134.000000,6.000000,135.000000,5.000000,64.463454,13.000000,35.264657
max,180.000000,9.000000,180.000000,6.000000,100.000000,20.000000,92.782669


In [6]:
df.STORE_NUMBER.nunique()

180

In [7]:
df.HELPER.unique()

array([False,  True])

In [8]:
df.URBANITY.unique()

array(['U', 'S', 'X'], dtype=object)

In [9]:
df.ROUTE_ID.unique()

array([1, 2, 5, 4, 6, 7, 9, 8, 3])

In [10]:
df.groupby('ROUTE_ID')['UNLOAD_TIME'].median()

ROUTE_ID
1    28.873604
2    29.393054
3    28.722521
4    29.249332
5    28.777764
6    28.910702
7    29.489292
8    28.883358
9    29.331447
Name: UNLOAD_TIME, dtype: float64

In [11]:
df.groupby('ROUTE_ID')['UNLOAD_TIME'].mean()

ROUTE_ID
1    28.939330
2    29.525679
3    28.939662
4    29.269774
5    29.251740
6    28.973765
7    29.304929
8    29.053279
9    29.280964
Name: UNLOAD_TIME, dtype: float64

In [12]:
df['UNLOAD_TIME'].mean()

29.163737173991052

In [13]:
df.groupby('STORE_NUMBER')['UNLOAD_TIME'].median().reset_index().sort_values('UNLOAD_TIME',ascending=False).head(10)

,STORE_NUMBER,UNLOAD_TIME
128,129,36.384452
61,62,35.560755
140,141,35.360932
107,108,35.164501
156,157,35.161524
149,150,34.985500
103,104,34.777445
113,114,34.761500
162,163,34.668502
26,27,34.624286


In [14]:
df.groupby('STORE_NUMBER')['UNLOAD_TIME'].mean().reset_index().sort_values('UNLOAD_TIME',ascending=False).head(10)

,STORE_NUMBER,UNLOAD_TIME
62,63,35.033952
149,150,34.953348
89,90,34.916043
103,104,34.884413
115,116,34.394336
76,77,34.348347
47,48,34.309766
121,122,34.251856
128,129,34.242381
69,70,34.136947


In [15]:
df.groupby('URBANITY')['UNLOAD_TIME'].median().reset_index().sort_values('UNLOAD_TIME',ascending=False).head(10)

,URBANITY,UNLOAD_TIME
1,U,33.264322
0,S,27.065998
2,X,20.842417


In [16]:
df.groupby('URBANITY')['UNLOAD_TIME'].mean().reset_index().sort_values('UNLOAD_TIME',ascending=False).head(10)

,URBANITY,UNLOAD_TIME
1,U,32.868562
0,S,26.625130
2,X,20.597517


In [17]:
df["avg_wt_of_case"] = df["TOTAL_WEIGHT"]/df["CASES"]

In [18]:
df.head()

,DAY_ID,ROUTE_ID,STORE_NUMBER,DAY_OF_WEEK,HELPER,TOTAL_WEIGHT,CASES,URBANITY,UNLOAD_TIME,avg_wt_of_case
0,1,1,176,1,False,76.759312,16,U,46.579371,4.797457
1,4,1,176,4,False,54.218237,11,U,32.698010,4.928931
2,5,2,176,5,False,80.911549,17,U,49.060463,4.759503
3,6,5,176,6,False,65.970720,14,U,35.219056,4.712194
4,8,4,176,1,False,60.334548,13,U,38.351855,4.641119


In [19]:
df['avg_wt_of_case'].min(), df['avg_wt_of_case'].max(), 

(3.7684276927040585, 5.0)

In [20]:
df['avg_wt_of_case'].min()

3.7684276927040585

In [21]:
df['range_avg_wt_of_case'] = pd.cut(df['avg_wt_of_case'], [3.75,4,4.25, 4.5, 4.75, 5], include_lowest=True)

In [22]:
df.head()

,DAY_ID,ROUTE_ID,STORE_NUMBER,DAY_OF_WEEK,HELPER,TOTAL_WEIGHT,CASES,URBANITY,UNLOAD_TIME,avg_wt_of_case,range_avg_wt_of_case
0,1,1,176,1,False,76.759312,16,U,46.579371,4.797457,"(4.75, 5.0]"
1,4,1,176,4,False,54.218237,11,U,32.698010,4.928931,"(4.75, 5.0]"
2,5,2,176,5,False,80.911549,17,U,49.060463,4.759503,"(4.75, 5.0]"
3,6,5,176,6,False,65.970720,14,U,35.219056,4.712194,"(4.5, 4.75]"
4,8,4,176,1,False,60.334548,13,U,38.351855,4.641119,"(4.5, 4.75]"


In [23]:
df.groupby('range_avg_wt_of_case')['UNLOAD_TIME'].mean().reset_index().sort_values('UNLOAD_TIME',ascending=False).head(10)

,range_avg_wt_of_case,UNLOAD_TIME
4,"(4.75, 5.0]",30.559943
3,"(4.5, 4.75]",27.776345
2,"(4.25, 4.5]",18.821518
1,"(4.0, 4.25]",12.667626
0,"(3.749, 4.0]",7.987277


In [24]:
df.groupby('range_avg_wt_of_case')['UNLOAD_TIME'].median().reset_index().sort_values('UNLOAD_TIME',ascending=False).head(10)

,range_avg_wt_of_case,UNLOAD_TIME
4,"(4.75, 5.0]",30.543798
3,"(4.5, 4.75]",27.737184
2,"(4.25, 4.5]",18.929165
1,"(4.0, 4.25]",11.755338
0,"(3.749, 4.0]",6.698664


In [25]:
df.groupby(['range_avg_wt_of_case','HELPER'])['UNLOAD_TIME'].mean().reset_index().sort_values('UNLOAD_TIME',ascending=False).head(10)

,range_avg_wt_of_case,HELPER,UNLOAD_TIME
8,"(4.75, 5.0]",False,32.075195
6,"(4.5, 4.75]",False,29.065329
4,"(4.25, 4.5]",False,19.616625
9,"(4.75, 5.0]",True,17.252715
7,"(4.5, 4.75]",True,15.666864
2,"(4.0, 4.25]",False,13.033101
5,"(4.25, 4.5]",True,11.205221
0,"(3.749, 4.0]",False,8.484097
3,"(4.0, 4.25]",True,7.916452
1,"(3.749, 4.0]",True,6.000000


In [26]:
df.groupby('DAY_OF_WEEK')['UNLOAD_TIME'].mean().reset_index().sort_values('UNLOAD_TIME',ascending=False).head(10)

,DAY_OF_WEEK,UNLOAD_TIME
4,4,29.341144
6,6,29.239998
2,2,29.194728
0,0,29.193222
1,1,29.146497
3,3,29.127350
5,5,28.913360


In [27]:
df.dtypes

DAY_ID                     int64
ROUTE_ID                   int64
STORE_NUMBER               int64
DAY_OF_WEEK                int64
HELPER                      bool
TOTAL_WEIGHT             float64
CASES                      int64
URBANITY                  object
UNLOAD_TIME              float64
avg_wt_of_case           float64
range_avg_wt_of_case    category
dtype: object

In [28]:
df.head()

,DAY_ID,ROUTE_ID,STORE_NUMBER,DAY_OF_WEEK,HELPER,TOTAL_WEIGHT,CASES,URBANITY,UNLOAD_TIME,avg_wt_of_case,range_avg_wt_of_case
0,1,1,176,1,False,76.759312,16,U,46.579371,4.797457,"(4.75, 5.0]"
1,4,1,176,4,False,54.218237,11,U,32.698010,4.928931,"(4.75, 5.0]"
2,5,2,176,5,False,80.911549,17,U,49.060463,4.759503,"(4.75, 5.0]"
3,6,5,176,6,False,65.970720,14,U,35.219056,4.712194,"(4.5, 4.75]"
4,8,4,176,1,False,60.334548,13,U,38.351855,4.641119,"(4.5, 4.75]"


## Train test split

In [31]:
X_cols = ["ROUTE_ID","DAY_OF_WEEK","HELPER","range_avg_wt_of_case","URBANITY"]
Y_cols = ["UNLOAD_TIME"]

In [32]:
X_train = df[X_cols]
y_train = df[Y_cols]

X_train.shape, y_train.shape

((15321, 5), (15321, 1))

In [33]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size = 0.2)

X_train.shape,X_test.shape,y_train.shape,y_test.shape

((12256, 5), (3065, 5), (12256, 1), (3065, 1))

In [73]:
X_test.head()

,ROUTE_ID,DAY_OF_WEEK,HELPER,range_avg_wt_of_case,URBANITY
2454,7,1,False,"(4.75, 5.0]",X
8723,5,6,False,"(4.75, 5.0]",U
14512,3,0,False,"(4.5, 4.75]",U
4276,2,1,False,"(4.5, 4.75]",U
7490,4,1,False,"(4.75, 5.0]",S


## Data preprocessing

In [34]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
d = defaultdict(LabelEncoder)

In [35]:
X_train.head()

,ROUTE_ID,DAY_OF_WEEK,HELPER,range_avg_wt_of_case,URBANITY
10045,2,6,False,"(4.75, 5.0]",U
569,5,6,False,"(4.75, 5.0]",S
11884,5,0,False,"(4.75, 5.0]",U
2777,2,5,False,"(4.5, 4.75]",S
12543,5,3,False,"(4.75, 5.0]",U


In [40]:
X_train_encoded = X_train.apply(lambda x: d[x.name].fit_transform(x))

In [42]:
X_test_encoded = X_test.apply(lambda x: d[x.name].transform(x))

In [41]:
X_train_encoded.head()

,ROUTE_ID,DAY_OF_WEEK,HELPER,range_avg_wt_of_case,URBANITY
10045,1,6,0,4,1
569,4,6,0,4,0
11884,4,0,0,4,1
2777,1,5,0,3,0
12543,4,3,0,4,1


## Modeling

In [48]:
regressor=xgb.XGBRegressor(eval_metric='rmsle')

In [50]:
%%time

from sklearn.model_selection import GridSearchCV
# set up our search grid
param_grid = {"max_depth":    [2, 4, 5],
              "n_estimators": [100, 300, 500],
              "learning_rate": [0.01, 0.001]}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train_encoded, y_train)

print("The best hyperparameters are ",search.best_params_)

The best hyperparameters are  {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 500}
CPU times: user 5min 25s, sys: 10.1 s, total: 5min 35s
Wall time: 31.4 s


In [52]:
regressor=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

regressor.fit(X_train_encoded, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [53]:
predictions = regressor.predict(X_test_encoded)

In [55]:
predictions[:5]

array([22.174187, 36.314472, 32.412415, 32.412415, 28.914831],
      dtype=float32)

In [56]:
from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(y_test, predictions) )
print("The score is %.5f" % RMSLE )

The score is 0.21712


## Error analysis

In [79]:
df_test_data_report = X_test[X_cols].copy()

In [80]:
df_test_data_report["actual_UNLOAD_TIME"] = y_test
df_test_data_report["predicted_UNLOAD_TIME"] = predictions

In [81]:
df_test_data_report.head()

,ROUTE_ID,DAY_OF_WEEK,HELPER,range_avg_wt_of_case,URBANITY,actual_UNLOAD_TIME,predicted_UNLOAD_TIME
2454,7,1,False,"(4.75, 5.0]",X,23.744685,22.174187
8723,5,6,False,"(4.75, 5.0]",U,36.655240,36.314472
14512,3,0,False,"(4.5, 4.75]",U,29.290374,32.412415
4276,2,1,False,"(4.5, 4.75]",U,28.525507,32.412415
7490,4,1,False,"(4.75, 5.0]",S,23.836401,28.914831


In [82]:
df_test_data_report = df_test_data_report.reset_index(drop=True)
df_test_data_report.head()

,ROUTE_ID,DAY_OF_WEEK,HELPER,range_avg_wt_of_case,URBANITY,actual_UNLOAD_TIME,predicted_UNLOAD_TIME
0,7,1,False,"(4.75, 5.0]",X,23.744685,22.174187
1,5,6,False,"(4.75, 5.0]",U,36.655240,36.314472
2,3,0,False,"(4.5, 4.75]",U,29.290374,32.412415
3,2,1,False,"(4.5, 4.75]",U,28.525507,32.412415
4,4,1,False,"(4.75, 5.0]",S,23.836401,28.914831


In [83]:
def error_fn(act, pred):
    return np.sqrt( mean_squared_log_error(act, pred))

In [84]:
df_test_data_report.groupby('URBANITY').apply(lambda x: error_fn(x['actual_UNLOAD_TIME'], x['predicted_UNLOAD_TIME']))

URBANITY
S    0.212841
U    0.218871
X    0.223119
dtype: float64

In [85]:
df_test_data_report.groupby('range_avg_wt_of_case').apply(lambda x: error_fn(x['actual_UNLOAD_TIME'], x['predicted_UNLOAD_TIME']))

range_avg_wt_of_case
(3.749, 4.0]    0.362361
(4.0, 4.25]     0.091924
(4.25, 4.5]     0.191933
(4.5, 4.75]     0.219113
(4.75, 5.0]     0.216961
dtype: float64

In [86]:
df_test_data_report.groupby(['HELPER','range_avg_wt_of_case']).apply(lambda x: error_fn(x['actual_UNLOAD_TIME'], x['predicted_UNLOAD_TIME']))

HELPER  range_avg_wt_of_case
False   (3.749, 4.0]            0.449593
        (4.0, 4.25]             0.091924
        (4.25, 4.5]             0.188621
        (4.5, 4.75]             0.221825
        (4.75, 5.0]             0.217857
True    (3.749, 4.0]            0.245922
        (4.25, 4.5]             0.250984
        (4.5, 4.75]             0.191206
        (4.75, 5.0]             0.208572
dtype: float64

In [87]:
df_test_data_report.groupby('DAY_OF_WEEK').apply(lambda x: error_fn(x['actual_UNLOAD_TIME'], x['predicted_UNLOAD_TIME']))

DAY_OF_WEEK
0    0.204930
1    0.225405
2    0.216463
3    0.209215
4    0.217346
5    0.222500
6    0.222404
dtype: float64

In [88]:
df_test_data_report.groupby('ROUTE_ID').apply(lambda x: error_fn(x['actual_UNLOAD_TIME'], x['predicted_UNLOAD_TIME']))

ROUTE_ID
1    0.225130
2    0.210055
3    0.206313
4    0.217845
5    0.224080
6    0.232540
7    0.204073
8    0.215739
9    0.190808
dtype: float64

## Other models experiment

In [90]:
from sklearn.linear_model import LinearRegression


LR = LinearRegression().fit(X_train_encoded, y_train)


predictions_LR = LR.predict(X_test_encoded)

RMSLE_LR = np.sqrt( mean_squared_log_error(y_test, predictions_LR) )
print("The score is %.5f" % RMSLE_LR )

The score is 0.26840


In [94]:
from sklearn.ensemble import RandomForestRegressor

rf_regr = RandomForestRegressor(max_depth=2, random_state=0)

rf_regr.fit(X_train_encoded, y_train)


predictions_rf = rf_regr.predict(X_test_encoded)

RMSLE_rf = np.sqrt( mean_squared_log_error(y_test, predictions_rf) )
print("The score is %.5f" % RMSLE_rf )


/var/folders/1v/r51qd_7n69zc85cfv183jxmw0000gn/T/ipykernel_3704/4205754115.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_regr.fit(X_train_encoded, y_train)


The score is 0.25445
